In [1]:
import os
from pathlib import Path

### Documnet folder

In [2]:
data_path = f"{Path.cwd().parent}/data_files/"
print(data_path)

/Users/ritkumar17/Desktop/RD_Project/agentic_learning/data_files/


In [3]:
file_path = f"{data_path}attention.pdf" 
print(file_path)

/Users/ritkumar17/Desktop/RD_Project/agentic_learning/data_files/attention.pdf


In [4]:
from docling.document_converter import DocumentConverter

In [5]:
source = file_path
converter = DocumentConverter()
doc = converter.convert(source).document

2026-01-24 17:44:15,724 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2026-01-24 17:44:15,774 - INFO - Going to convert document batch...
2026-01-24 17:44:15,775 - INFO - Initializing pipeline for StandardPdfPipeline with options hash e15bc6f248154cc62f8db15ef18a8ab7
2026-01-24 17:44:15,781 - INFO - Loading plugin 'docling_defaults'
2026-01-24 17:44:15,783 - INFO - Registered picture descriptions: ['vlm', 'api']
2026-01-24 17:44:15,788 - INFO - Loading plugin 'docling_defaults'
2026-01-24 17:44:15,791 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2026-01-24 17:44:16,591 - INFO - Auto OCR model selected ocrmac.
2026-01-24 17:44:16,597 - INFO - Loading plugin 'docling_defaults'
2026-01-24 17:44:16,599 - INFO - Registered layout engines: ['docling_layout_default', 'docling_experimental_table_crops_layout']
2026-01-24 17:44:16,602 - INFO - Accelerator device: 'mps'
2026-01-24 17:44:22,723 - INFO - Loading plugin 'docling_defaul

In [6]:
import json
print(json.dumps(doc.export_to_dict(), indent=2))

{
  "schema_name": "DoclingDocument",
  "version": "1.8.0",
  "name": "attention",
  "origin": {
    "mimetype": "application/pdf",
    "binary_hash": 1511601997122043754,
    "filename": "attention.pdf"
  },
  "furniture": {
    "self_ref": "#/furniture",
    "children": [],
    "content_layer": "furniture",
    "name": "_root_",
    "label": "unspecified"
  },
  "body": {
    "self_ref": "#/body",
    "children": [
      {
        "$ref": "#/texts/0"
      },
      {
        "$ref": "#/texts/1"
      },
      {
        "$ref": "#/texts/2"
      },
      {
        "$ref": "#/texts/3"
      },
      {
        "$ref": "#/texts/4"
      },
      {
        "$ref": "#/texts/5"
      },
      {
        "$ref": "#/texts/6"
      },
      {
        "$ref": "#/texts/7"
      },
      {
        "$ref": "#/texts/8"
      },
      {
        "$ref": "#/texts/9"
      },
      {
        "$ref": "#/texts/10"
      },
      {
        "$ref": "#/texts/11"
      },
      {
        "$ref": "#/texts/12"


In [7]:
print(doc.export_to_markdown()[:1000]) 

Provided proper attribution is provided, Google hereby grants permission to reproduce the tables and figures in this paper solely for use in journalistic or scholarly works.

## Attention Is All You Need

Ashish Vaswani ∗ Google Brain avaswani@google.com

Noam Shazeer ∗ Google Brain noam@google.com

Llion Jones ∗ Google Research llion@google.com

Niki Parmar ∗ Google Research nikip@google.com

Aidan N. Gomez ∗ † University of Toronto aidan@cs.toronto.edu

Jakob Uszkoreit ∗ Google Research usz@google.com

Łukasz Kaiser ∗ Google Brain lukaszkaiser@google.com

∗ ‡

Illia Polosukhin illia.polosukhin@gmail.com

## Abstract

The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and 

In [8]:
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import (
    PdfPipelineOptions,
    PictureDescriptionApiOptions
)
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling_core.types.doc.document import ImageRefMode

def convert_with_image_annotation(input_doc_path):
    api_key = os.environ.get("OPENAI_API_KEY")
    model = "gpt-4.1-mini"
    picture_desc_api_option = PictureDescriptionApiOptions(
        url="https://api.openai.com/v1/chat/completions",
        prompt="Describe this image in sentences in a single paragraph.",
        params=dict(
                model=model,
            ),
        headers={
            "Authorization": "Bearer " + api_key,
        },
        timeout=60,
    )

    pipeline_options = PdfPipelineOptions(
        do_picture_description=True,
        picture_description_options=picture_desc_api_option,
        enable_remote_services=True,
        generate_picture_images=True,
        images_scale=2,
        do_ocr=False
    )

    converter = DocumentConverter(
        format_options={InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)}
    )
    conv_res = converter.convert(source=input_doc_path)
    return conv_res

In [9]:
result = convert_with_image_annotation(file_path)

2026-01-24 17:44:43,108 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2026-01-24 17:44:43,110 - INFO - Going to convert document batch...
2026-01-24 17:44:43,111 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 140c47e22cf4377426094d6b433dfe4e
2026-01-24 17:44:43,111 - INFO - Accelerator device: 'mps'
2026-01-24 17:44:43,936 - INFO - Accelerator device: 'mps'
2026-01-24 17:44:44,396 - INFO - Processing document attention.pdf
2026-01-24 17:45:08,980 - INFO - Finished converting document attention.pdf in 25.87 sec.


In [10]:
print(result.document.export_to_markdown(mark_annotations = True, include_annotations=True))

Provided proper attribution is provided, Google hereby grants permission to reproduce the tables and figures in this paper solely for use in journalistic or scholarly works.

## Attention Is All You Need

Ashish Vaswani ∗ Google Brain avaswani@google.com

Noam Shazeer ∗ Google Brain noam@google.com

Llion Jones ∗ Google Research llion@google.com

Niki Parmar ∗ Google Research nikip@google.com

Aidan N. Gomez ∗ † University of Toronto aidan@cs.toronto.edu

Jakob Uszkoreit ∗ Google Research usz@google.com

Łukasz Kaiser ∗ Google Brain lukaszkaiser@google.com

∗ ‡

Illia Polosukhin illia.polosukhin@gmail.com

## Abstract

The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and 

In [11]:
def export_function_md_with_image_ref(conv_res, output_path:str, replace_blank:str="_"):
    
    output_dir = Path(output_path)
    output_dir.mkdir(parents=True, exist_ok=True)
    doc_filename = conv_res.input.file.stem.replace(" ", replace_blank)

    # Save markdown with externally referenced pictures
    md_filename = output_dir / f"{doc_filename}-with-image-refs_noc.md"
    conv_res.document.save_as_markdown(md_filename, image_mode=ImageRefMode.REFERENCED, include_annotations=True)

In [12]:
output_path = f"{data_path}docling_output"

export_function_md_with_image_ref(result, output_path)